<a href="https://colab.research.google.com/github/lily-880/Bear-classifier/blob/main/bear_classifier_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from fastai.vision.all import *
from pathlib import Path
import urllib.request


In [ ]:
from fastai.vision.all import *
from pathlib import Path
import urllib.request, zipfile

MODEL_URL = "https://github.com/lily-880/Bear-classifier/releases/download/v1/export.pkl"
DOWNLOADED = Path("export.pkl")          # 实际是zip
EXTRACT_DIR = Path("model_unzipped")
INNER_PKL = EXTRACT_DIR/"export"/"data.pkl"   # 你输出里已经显示它存在

if not DOWNLOADED.exists():
    print("Downloading model...")
    urllib.request.urlretrieve(MODEL_URL, DOWNLOADED)

head2 = DOWNLOADED.read_bytes()[:2]
print("File header:", head2)

# 如果是zip（PK开头），就解压
if head2 == b"PK":
    print("Looks like a zip, extracting...")
    EXTRACT_DIR.mkdir(exist_ok=True)
    with zipfile.ZipFile(DOWNLOADED, "r") as z:
        z.extractall(EXTRACT_DIR)
    model_path = INNER_PKL
else:
    model_path = DOWNLOADED

print("Loading learner from:", model_path)
learn = load_learner(model_path)
print("Learner loaded OK.")

In [ ]:
import ipywidgets as widgets
from IPython.display import display

uploader = widgets.FileUpload(accept='image/*', multiple=False)
out = widgets.Output()

def on_upload_change(change):
    out.clear_output()
    with out:
        img = PILImage.create(list(uploader.value.values())[0]['content'])
        pred, pred_idx, probs = learn.predict(img)
        display(img.to_thumb(256))
        print(f"Prediction: {pred}")
        print(f"Confidence: {probs[pred_idx]:.3f}")

uploader.observe(on_upload_change, names='value')

display(uploader, out)
